ref: https://github.com/sakunaharinda/ragatouille-book/blob/main/book/2_Query_Transformation.ipynb

In [1]:
from dotenv import load_dotenv
load_dotenv()
import rich

In [19]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

examples = [
    {
        'input': 'What happens to the pressure, P, of an ideal gas if the temperature is increased by a factor of 2 and the volume is increased by a factor of 8?',
        'output': 'What are the physics principles behind this question?'
    },
    {
        'input': 'Estella Leopold went to which school between Aug 1954 and Nov 1954?',
        'output': "What was Estella Leopold's education history?"
    }
]

example_prompt = ChatPromptTemplate.from_messages(
        [
            ('human', '{input}'), 
            ('ai', '{output}')
        ]
    )

few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=example_prompt
)

In [6]:
# Test
rich.print(few_shot_prompt.format())
rich.print(few_shot_prompt)

Human: What happens to the pressure, P, of an ideal gas if the temperature is increased by a factor of 2 and the 
volume is increased by a factor of 8?
AI: What are the physics principles behind this question?
Human: Estella Leopold went to which school between Aug 1954 and Nov 1954?
AI: What was Estella Leopold's education history?

FewShotChatMessagePromptTemplate(
    examples=[
        {
            'input': 'What happens to the pressure, P, of an ideal gas if the temperature is increased by a factor 
of 2 and the volume is increased by a factor of 8?',
            'output': 'What are the physics principles behind this question?'
        },
        {
            'input': 'Estella Leopold went to which school between Aug 1954 and Nov 1954?',
            'output': "What was Estella Leopold's education history?"
        }
    ],
    input_variables=[],
    input_types={},
    partial_variables={},
    example_prompt=ChatPromptTemplate(
        input_variables=['input', 'output'],
        input_types={},
        partial_variables={},
        messages=[
            HumanMessagePromptTemplate(
                prompt=PromptTemplate(
                    input_variables=['input'],
                    input_types={},
                    partial_variables={},
                    template='{input}'
                ),
                additional_kwargs={}
            ),
            AIMessagePromptTemplate(
                prompt=PromptTemplate(
                    input_variables=['output'],
                    input_types={},
                    partial_variables={},
                    template='{output}'
                ),
                additional_kwargs={}
            )
        ]
    )
)

In [11]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ('system', """You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:"""),
        few_shot_prompt,
        ('user', '{question}')
    ]
)

In [10]:
rich.print(final_prompt.format(question= "What need to consider when using LLM to eval LLM generation?"))

System: You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic
step-back question, which is easier to answer. Here are a few examples:
Human: What happens to the pressure, P, of an ideal gas if the temperature is increased by a factor of 2 and the 
volume is increased by a factor of 8?
AI: What are the physics principles behind this question?
Human: Estella Leopold went to which school between Aug 1954 and Nov 1954?
AI: What was Estella Leopold's education history?
Human: What need to consider when using LLM to eval LLM generation?

#### There are few ways you can write the chain
##### Method 1

In [18]:
step_back_query_chain = (
    final_prompt
    | ChatOpenAI(model="gpt-4o-mini", temperature=0.8)
    | StrOutputParser()
)

step_back_query_chain.invoke({"question": "What need to consider when using LLM to eval LLM generation?"})

'What factors should be taken into account when evaluating the output of language models?'

In [20]:
step_back_query_chain = (
    {"question": RunnablePassthrough()}
    | final_prompt
    | ChatOpenAI(model="gpt-4o-mini", temperature=0.8)
    | StrOutputParser()
)

step_back_query_chain.invoke({"question": "What need to consider when using LLM to eval LLM generation?"})

'What factors should be taken into account when evaluating the output of language models?'

我們需要用step_back_prompt找回的文件，以及原本query的文件來做出最後的回答

In [21]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [22]:
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/Users/sean_liu/miniconda3/envs/langchain/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/Users/sean_liu/miniconda3/envs/langchain/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [24]:
loader = DirectoryLoader('../../pdf_files/',glob="*.pdf",loader_cls=PyPDFLoader)
documents = loader.load()

# Split text into chunks

text_splitter  = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
text_chunks = text_splitter.split_documents(documents)

vectorstore = Chroma.from_documents(documents=text_chunks, 
                                    embedding=embedding,
                                    persist_directory="data/vectorstore")
vectorstore.persist()

retriever = vectorstore.as_retriever()

In [28]:
response_prompt_template = """You are an expert of world knowledge. 
I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. 
Otherwise, ignore them if they are not relevant.

<normal_context>
# {normal_context}
</normal_context>

<step_back_context>
# {step_back_context}
</step_back_context>


# Original Question: {question}
# Answer:"""

response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

# Either syntax 1 or syntax 2 works, since only one input "question" is passed to the chain
# Syntax 1
# step_back_and_response_chain = ({"normal_context": RunnablePassthrough() |  retriever,
#      "step_back_context": RunnablePassthrough() | step_back_query_chain | retriever,
#      "question": RunnablePassthrough()}
#      | response_prompt
#      | ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
#      | StrOutputParser()
# )

# Syntax 2
step_back_and_response_chain = (
    {"question": RunnablePassthrough()}
    | {"normal_context": RunnablePassthrough() |  retriever,
     "step_back_context": RunnablePassthrough() | step_back_query_chain | retriever,
     "question": RunnablePassthrough()}
     | response_prompt
     | ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
     | StrOutputParser()
)

In [29]:
res = step_back_and_response_chain.invoke("What need to consider when using LLM to eval LLM generation?")

In [30]:
rich.print(res)

When using large language models (LLMs) to evaluate the outputs generated by other LLMs, several important 
considerations should be taken into account:

1. **Reliability of Evaluators**: LLMs are increasingly being used as proxies for human evaluators. However, their 
reliability in this role is still under scrutiny. It is crucial to understand that LLMs can be sensitive to the 
specific textual instructions and inputs they receive, which may affect their evaluation outcomes.

2. **Evaluation Pipeline Design**: The design of the evaluation pipeline is essential. For instance, systems like 
Vicuna utilize LLMs (e.g., GPT-4) to score and compare candidate responses while providing explanations. The 
simplicity and interpretability of such pipelines can be beneficial, but the underlying assumptions about the LLM's
evaluative capabilities must be critically assessed.

3. **Bias and Fairness**: LLMs may exhibit biases based on the data they were trained on. This can lead to unfair 
evaluations, especially if the evaluation criteria are not carefully defined or if the LLMs are not adequately 
calibrated to recognize diverse perspectives and contexts.

4. **Context Sensitivity**: The performance of LLMs as evaluators can vary significantly based on the context in 
which they are used. Factors such as the phrasing of prompts, the nature of the responses being evaluated, and the 
specific evaluation criteria can all influence the results.

5. **Interpretability of Results**: While LLMs can provide explanations for their evaluations, the interpretability
of these explanations is critical. Users should be cautious about relying on LLM-generated evaluations without a 
clear understanding of how those evaluations were derived.

6. **Comparative Evaluation**: It may be beneficial to compare LLM evaluations with human evaluations or other 
established metrics to gauge their effectiveness and reliability. This can help identify any discrepancies and 
improve the evaluation process.

7. **Continuous Improvement**: As LLMs evolve, so too should the methods used for evaluation. Regular updates and 
refinements to the evaluation criteria and processes are necessary to ensure that they remain relevant and 
effective.

By considering these factors, users can better navigate the complexities involved in using LLMs for evaluating 
LLM-generated content, ultimately leading to more reliable and fair assessments.